In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-02-26 20:06:56.730869: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-02-26 20:07:18.245760: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 20:07:18.300648: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-26 20:07:18.422775: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-02-26 20:07:18.747689: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-26 20:07:18.747735: I tensorflow/stream_executo

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']
cat_vars = ['Store','Year','Month','StateHoliday','CompetitionMonthsOpen',
    'Promo2Weeks','StoreType','Assortment','PromoInterval',
    'CompetitionOpenSinceYear','State',
    'Week','Events','Promo_fw','Promo_bw','StateHoliday_bool_fw', 'StateHoliday_bool_bw','SchoolHoliday_fw','SchoolHoliday_bw']#'DayOfWeek'

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [5]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
 'CloudCover', 'trend',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']
cat_vars = ['Store', 'DayOfWeek', 'Promo_bw', 'Month', 'Year', 'State', 'Assortment', 'StoreType', 'Events', 'StateHoliday', 'Promo2SinceYear']
contin_vars = [ 'Max_TemperatureC', 'Precipitationmm', 'Promo', 'SchoolHoliday', 'trend', 'Mean_TemperatureC','CompetitionDistance']
# contin_vars = []

# cat_vars = ['Store',
#   'DayOfWeek',
#   'Year',
#   'Month',
#   'Day',
#   'StateHoliday',
#   'CompetitionMonthsOpen',
#   'Promo2Weeks',
#   'StoreType',
#   'Assortment',
#   'PromoInterval',
#   'CompetitionOpenSinceYear',
#   'Promo2SinceYear',
#   'State',
#   'Week',
#   'Events',
#   'Promo_fw',
#   'Promo_bw',
#   'SchoolHoliday_fw',
#   'SchoolHoliday_bw']
# contin_vars = ['CompetitionDistance', 
#    'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
#    'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#    'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#    'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [6]:
y_out_columns = ['Sales']

In [7]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [8]:
X_train = df_train[cat_vars + contin_vars]
X_valid = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [9]:
X_train.shape, X_valid.shape

((814150, 18), (30188, 18))

In [10]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

/home/facu/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


## Categorical columns transformation

In [11]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'


In [12]:
# from sklearn.model_selection import TimeSeriesSplit
# n=len(df.Sales)

# train_ratio=0.9
# train_size = int(n*train_ratio)
# # contins_trn, contins_val = df[contin_vars].loc[:train_size-1], df[contin_vars].loc[train_size:]
# # cats_trn, cats_val       = df[cat_var_dict.keys()].loc[:train_size-1], df[cat_var_dict.keys()].loc[train_size:]

# X_train,X_valid                  = df[cat_vars+contin_vars].loc[:train_size-1], df[cat_vars+contin_vars].loc[train_size:]

# max_y_log                        = (np.log(df.Sales)).values.max()
# y_train, y_valid                 = np.log(df.Sales.loc[:train_size-1])/max_y_log, np.log(df.Sales.loc[train_size:])/max_y_log

# print("X_train[",type(X_train),"]: ",X_train.shape,"      y_train[",type(y_train),"]: ",y_train.shape)
# print("X_valid[",type(X_train),"]: ",X_valid.shape,"      y_valid[",type(y_valid),"]: ",y_valid.shape)

# tscv = TimeSeriesSplit(n_splits=2, test_size=(n - train_size))
# display('test_size: ',(n - train_size))

In [13]:
if categoricals_processing == 'use_embeddings':
    embeddings_model = load_model('embeddings_model.hdf5')
    # display(embeddings_model.summary())
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
    X_valid = embeddings_model.predict(np.hsplit(X_valid, X_valid.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

2023-02-26 20:07:26.332234: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-26 20:07:26.333370: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


2023-02-26 20:07:27.679349: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-02-26 20:07:27.796111: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2599990000 Hz


25443/25443 [==============================] - 25s 916us/step


2023-02-26 20:07:52.411455: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 283324200 exceeds 10% of free system memory.


1284/1284 [==============================] - 3s 2ms/step


In [14]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    X_valid = ohe.transform(X_valid)
    X_test = ohe.transform(X_test)

In [15]:
X_train.shape

(814150, 87)

In [16]:
X_valid.shape

(30188, 87)

In [17]:
X_test.shape

(41088, 87)

## XGBoost

In [18]:
# n=len(df.Sales)

# train_ratio=0.9
# train_size = int(n*train_ratio)
# tscv = TimeSeriesSplit(n_splits=2, test_size=(n - train_size))
# for train_index, test_index in tscv.split(df[cat_vars+contin_vars]):
#     print("-----------")
#     print(train_index)
#     print(test_index)

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import get_scorer

In [20]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [21]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(((y_true - y_pred)/y_true)**2))

In [22]:
# parameters = {
#     'n_estimators'      :[3500,4500],
#     'eta'               :[0.1,0.2], 
#     'objective'         :['reg:squarederror'],
#     'n_jobs'            :[4],
#     'max_depth'         :[3,5],
#     'subsample'         :[0.8],#[0.2,0.8],
#     'colsample_bytree'  :[0.8],#[0.2,0.8],
#     'alpha'             :[0.2,0.3,0.5],
#     'gamma'             :[0]
#                     }
# parameters   = { 'n_estimators'      : 4000,
#                  'learning_rate'     : 0.2, 
#                  'max_depth'         : 13,
#                  'max_leaves'        : 0, #DEFAULT: 0 (unlimited)
#                  'gamma'             : 0.004,
#                  'reg_alpha'         : 0.07,
#                  #'reg_lambda'         : 0.2,
#                }

# grid_search_rmspe = make_scorer(rmspe,greater_is_better=False)
# model = XGBRegressor()

In [23]:
# clf = GridSearchCV(model, parameters, n_jobs = 4, verbose=100, cv=2, scoring=grid_search_rmspe)

In [37]:
n_estimators= 3000
learning_rate=0.2
model = XGBRegressor(n_estimators=n_estimators,
                     eta=learning_rate, 
                     objective='reg:squarederror',
                     n_jobs=-1,
                     max_depth=13,
                     min_child_weight = 1,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     alpha=0.07,
                     gamma=0.005
                    )
model

XGBRegressor(alpha=0.07, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, early_stopping_rounds=None,
             enable_categorical=False, eta=0.2, eval_metric=None,
             feature_types=None, gamma=0.005, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=13,
             max_leaves=None, min_child_weight=1, missing=nan,
             monotone_constraints=None, n_estimators=3000, n_jobs=-1,
             num_parallel_tree=None, ...)

In [38]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            # "eval_set" : [(X_valid, y_valid)],
            "eval_set" : [(X_valid, y_val.reshape(-1))],
            'verbose': 100,
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

/home/facu/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/facu/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:0.26287
[100]	validation_0-rmse:0.01257
[200]	validation_0-rmse:0.01240
[300]	validation_0-rmse:0.01233
[400]	validation_0-rmse:0.01228
[500]	validation_0-rmse:0.01228
[600]	validation_0-rmse:0.01224
[700]	validation_0-rmse:0.01222


# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [29]:
#model = clf

In [31]:
score = model.score(X_valid, y_val)

In [32]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_valid)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [33]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [34]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [35]:
print(score, train_RMSE, val_RMSE)

0.886748132936197 0.09842086748103157 0.13625017955745045


# Baseline

In [ ]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [ ]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [ ]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [ ]:
df_test[['Store', 'Sales']].head(10)

In [ ]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

In [ ]:
sample_csv['Sales'] = df_test['Sales']

In [ ]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [ ]:
sample_csv.head()

# Sumbit a la competición

In [36]:
sample_csv = pd.read_csv('rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_XGBoost.csv', index=False)